### Solution of the Wave Equation on the Line Using a PINN

The wave equation is:

$$
u_{tt} - a u_{xx} = 0, \quad (t,x) \in [0,T] \times [0,X]
$$

with initial conditions:

$$
u(0, x) = \sin(k x)
$$

$$
u_t(0, x) = 0, \quad x \in [0, X]
$$

and boundary conditions:

$$
u(t, 0) = 0, \quad t \in [0, T]
$$

$$
u(t, X) = \sin(k X) \cos(\sqrt{a} k t), \quad t \in [0, T]
$$


In [ ]:
%matplotlib widget

import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML
from matplotlib.animation import FuncAnimation
from matplotlib.lines import Line2D
from torch import tensor

from examples.equations.wave_1d.pde import reference_solution
from measure_uq.models import PINN, PINN_PCE
from measure_uq.pde import PDE
from measure_uq.trainers.trainer import Trainer
from measure_uq.utilities import cartesian_product_of_rows, to_numpy

if 0:
    model_type = PINN_PCE
    model_path = "data/best_model_pinn_pce.pickle"
    pde_path = "data/pde_pinn_pce.pickle"
    trainer_path = "data/trainer_pinn_pce.pickle"
else:
    model_type = PINN
    model_path = "data/best_model_pinn.pickle"
    pde_path = "data/pde_pinn.pickle"
    trainer_path = "data/trainer_pinn.pickle"

model = model_type.load(model_path)
pde = PDE.load(pde_path)

trainer = Trainer.load(trainer_path)

T = pde.conditions_train.conditions[0].T
X = pde.conditions_train.conditions[0].X

t = tensor(np.linspace(0, T, 101)[:, None])
x = tensor(np.linspace(0, X, 51)[:, None])

# pde.parameters_test.N = 100
# pde.parameters_test.sample_values()
parameters = pde.parameters_test.values.detach().cpu()

Nx = x.shape[0]
Nt = t.shape[0]
Np = parameters.shape[0]

### Evaluate the solution of the PINN for all the parameters

In [ ]:
tx = cartesian_product_of_rows(t, x).float()

_, y_tmp = model.forward(tx, parameters)

y = np.array(
    [y_tmp[k::Np, :].reshape(Nt, Nx).detach().numpy().T for k in range(Np)],
)  # shape: (samples, Nx, Nt)

mean_y = y.mean(axis=0)
std_y = y.std(axis=0)

### Evaluate the reference solution for all the parameters


In [ ]:
tt = to_numpy(t).squeeze()
xx = to_numpy(x).squeeze()

s_tmp = [reference_solution(tt, xx, to_numpy(p)) for p in parameters]

solutions = np.stack(s_tmp, axis=0)  # shape: (samples, Nx, Nt)
mean_solution = solutions.mean(axis=0)
std_solution = solutions.std(axis=0)

### Plot the PINN and the reference solution

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(16, 10))
axs = axs.flatten()

axs[0].set_ylim(1.05 * y.min(), 1.05 * y.max())
axs[1].set_ylim(
    1.05 * (mean_y.min() - std_y.max()), 1.05 * (mean_y.max() + std_y.max())
)

i = 0

ax0_lines1 = axs[0].plot(x, solutions[:, :, i].T)
ax0_lines2 = axs[0].plot(x, y[:, :, i].T, "--")

(ax1_sol_mean,) = axs[1].plot(
    x,
    mean_solution[:, i],
    "-",
    color="black",
    label="MC mean ",
)
(ax1_sol_std_plus,) = axs[1].plot(
    x,
    mean_solution[:, i] + std_solution[:, i],
    "--",
    color="black",
    label="MC mean + std",
)
(ax1_sol_std_minus,) = axs[1].plot(
    x,
    mean_solution[:, i] - std_solution[:, i],
    "--",
    color="black",
    label="MC mean - std",
)

(ax1_y_mean,) = axs[1].plot(
    x,
    mean_y[:, i],
    ".",
    color="blue",
    label="PINN mean ",
)

axs[1].fill_between(
    x.squeeze(),
    mean_y[:, i] - std_y[:, i],
    mean_y[:, i] + std_y[:, i],
    color="blue",
    alpha=0.3,
    label="PINN uncertainty band (1 std)",
)

for a in axs:
    a.grid()

fig.suptitle(f"t = {t[i].item():.2f}")
axs[1].legend()


def animate(
    i: int,
) -> tuple[Line2D, Line2D, Line2D, Line2D, Line2D, Line2D, Line2D]:
    """
    Animate the solution of the PDE.

    Parameters
    ----------
    i : int
        The index of the frame.

    Returns
    -------
    tuple[Line2D, Line2D, Line2D, Line2D, Line2D, Line2D, Line2D]
        The lines of the animation.
    """
    fig.suptitle(f"t = {t[i].item():.2f}")

    for k, line in enumerate(ax0_lines1):
        line.set_ydata(solutions[k, :, i].T)

    for k, line in enumerate(ax0_lines2):
        line.set_ydata(y[k, :, i].T)

    ax1_sol_mean.set_ydata(mean_solution[:, i])
    ax1_sol_std_plus.set_ydata(mean_solution[:, i] + std_solution[:, i])
    ax1_sol_std_minus.set_ydata(mean_solution[:, i] - std_solution[:, i])
    ax1_y_mean.set_ydata(mean_y[:, i])

    axs[1].collections[0].remove()

    ax1_fill = axs[1].fill_between(
        x.squeeze(),
        mean_y[:, i] - std_y[:, i],
        mean_y[:, i] + std_y[:, i],
        color="blue",
        alpha=0.3,
        label="Uncertainty band (1 std)",
    )

    return (
        ax0_lines1,
        ax0_lines2,
        ax1_y_mean,
        ax1_sol_mean,
        ax1_sol_std_plus,
        ax1_sol_std_minus,
        ax1_fill,
    )


anime = FuncAnimation(
    fig,
    animate,
    interval=200,
    blit=False,
    frames=t.shape[0],
)

plt.close()
HTML(anime.to_html5_video())

### Plot the train and test loss

In [ ]:
_, ax = plt.subplots()

x1 = trainer.trainer_data.losses_train.i
y1 = trainer.trainer_data.losses_train.v
ax.plot(x1, y1, label="Training loss")

x2 = trainer.trainer_data.losses_test.i
y2 = trainer.trainer_data.losses_test.v
ax.plot(x2, y2, label="Testing loss")

ax.set_yscale("log")
ax.grid(True)


ax.set_xlabel("Iteration")
ax.set_ylabel("Loss")
ax.legend()

plt.show()